# AADT Confidence Interval, SR-99 District 3

## Helpful Links

---

### FHWA Links
* Guidelines for Obtaining AADT Estimates from Non-Traditional Sources:
    * https://www.fhwa.dot.gov/policyinformation/travel_monitoring/pubs/aadtnt/Guidelines_for_AADT_Estimates_Final.pdf

---
  
### AADT Analysis Locations
* 10 locations were used in the analysis
* Locations were determined based on the location on installed & recording Traffic Operations cameras
    * for additional information contact Zhenyu Zhu with Traffic Operations

### Traffic Census Data
* https://dot.ca.gov/programs/traffic-operations/census/traffic-volumes
* Back AADT, Peak Month, and Peak Hour usually represents traffic South or West of the count location.  
* Ahead AADT, Peak Month, and Peak Hour usually represents traffic North or East of the count location. Listing of routes with their designated  

* Because the Back & Ahead counts are included at each location in the Traffic Census Data, (e.g., "IRWINDALE, ARROW HIGHWAY") only one [OBJECTID*] per location was pulled; for this analysis the North Bound Nodes were used for the analysis. 
    * for more information see the diagram: https://traffic.onramp.dot.ca.gov/downloads/traffic/files/performance/census/Back_and_Ahead_Leg_Traffic_Count_Diagram.pdf

### StreetLight Analysis Data
* Analysis Type == Network Performance
* Segment Metrics
* 2022 was used to match currently available Traffic Census Data (as of 8/27/2025)
* pulled a variety of Day Types, but plan to just look at """All Day Types"""
* pulled a variety of Day Parts, but plan to just look at """All Day Parts"""

---


## Identify the corridor

In [1]:
# pull in the coordinates from the utils docs
#from osow_frp_o_d_utils_v3 import origin_intersections, destination_intersections
import shs_ct_tc_locations_utils as tc_locs

In [2]:
# Available corridors
    # "interstate_605_d7_tc_aadt_locations"
    # "sr_99_d3_tc_aadt_locations"

### Update the corridor name

In [3]:
# Identify the corridor to be analyzed
#CORRIDOR_VAR_NAME = "interstate_605_d7_tc_aadt_locations"
CORRIDOR_VAR_NAME = "sr_99_d3_tc_aadt_locations"

In [4]:
# Resolve the object from the module by name
try:
    aadt_locations = getattr(tc_locs, CORRIDOR_VAR_NAME)
except AttributeError:
    raise KeyError(
        f"'{CORRIDOR_VAR_NAME}' not found in shs_ct_tc_locations_utils. "
        "Double-check the variable name."
    )

## import packages

In [5]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import t as student_t  # if SciPy is not available, use a small lookup table
import csv
import re
import unicodedata

from pathlib import Path

## Step 0, Pull in the Data

In [6]:
# ------------------------------------------------------
# 0) Pull in the Data
# ------------------------------------------------------

# This function will pull in the data and clean the column headers in a way that will make them easier to work with
def getdata_and_cleanheaders(path):
    # Read the CSV file
    df = pd.read_csv(path)

    # Clean column headers: remove spaces, convert to lowercase, and strip trailing asterisks
    cleaned_columns = []
    for column in df.columns:
        cleaned_column = column.replace(" ", "").lower().rstrip("*")
        cleaned_columns.append(cleaned_column)

    df.columns = cleaned_columns
    return df

### import option 0.1: Identify the Google Cloud Storage path

In [7]:
# Identify the GCS path to the data
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/compare_traffic_counts/0_2022/"

In [8]:
#pull in the data & create dataframes
df_tc = getdata_and_cleanheaders(f"{gcs_path}caltrans_traffic_census_2022.csv")  # Traffic Census

In [9]:
# Available datasets
    # streetlight_605_d7_all_vehicles_np_2022.csv
    # streetlight_99_d3_all_vehicles_2022_np.csv

In [10]:
# Identify the StreetLight Analysis to be used in the AADT comparison
# df_stl = getdata_and_cleanheaders(f"{gcs_path}streetlight_605_d7_all_vehicles_np_2022.csv")  # StreetLight
df_stl = getdata_and_cleanheaders(f"{gcs_path}streetlight_99_d3_all_vehicles_2022_np.csv")  # StreetLight

### import option 0.2: Identify the local data path

In [11]:
# # Base data folder: aadt_confidence_interval/aadt_data/2022
# LOCAL_DATA_DIR = Path.cwd() / "aadt_data" / "2022"
# if not LOCAL_DATA_DIR.exists():
#     raise FileNotFoundError(f"Data folder not found: {LOCAL_DATA_DIR.resolve()}")

In [12]:
# # Traffic Census (traditional) — local CSV
# df_tc = getdata_and_cleanheaders(LOCAL_DATA_DIR / "caltrans_traffic_census_2022.csv")

In [13]:
# Available datasets
    # streetlight_605_d7_all_vehicles_np_2022.csv
    # streetlight_99_d3_all_vehicles_2022_np.csv

In [14]:
# # Non-Traditional (StreetLight) — local location
# df_stl = getdata_and_cleanheaders(LOCAL_DATA_DIR / "streetlight_605_d7_all_vehicles_np_2022.csv")
# df_stl = getdata_and_cleanheaders(LOCAL_DATA_DIR / "streetlight_99_d3_all_vehicles_2022_np.csv")

### Export to a CSV for viewing/validation

In [15]:
# export traditional dataset (Traffic Census
df_tc.to_csv("df_tc.csv", index=False)

# export non-traditional dataset (StreetLight)
df_stl.to_csv("df_stl.csv", index=False)

## Step 1, Build a per-location summary of Traffic Census locations

* **Goal:** For each location, pick the relevant **Traffic Census node** (objectid's), compute a representative **traditional AADT**, and emit a tidy per-location record.
* **OBJECTID selection policy**: Prefer **ODD** objectid's, if non available, use **all** available.
* **Outputs (one row per location):**
    * location, daytype, objectids, n_objectids, n_found_in_tc, missing_objectids, traditional_ahead_mean, traditional_behind_mean, traditional_aadt
* **Why this matters:** We now have a clean, comparable **traditional AADT** baseline per location to contrast with the non-traditional estimates in later steps

In [16]:
# ------------------------------------------------------------
# 1) Build a per-location summary of Traffic Census locations
# ------------------------------------------------------------


def traditional_aadt_by_location(aadt_locations, df_tc, as_df=True, use_parity=False):
    """
    Build a per-location summary of *traditional* (Traffic Census) AADT.

    Policy:
      • If multiple objectids exist for a location, prefer those whose numeric value is ODD.
        If no odd ids exist, fall back to all ids found.
      • Default behavior (use_parity=False): for each kept objectid, compute (ahead_aadt + back_aadt)/2,
        then average across kept objectids.
        
    Output columns:
      location, daytype, objectids, n_objectids, n_found_in_tc, missing_objectids,
      traditional_ahead_mean, traditional_behind_mean, traditional_aadt
    """
    # Requires: import pandas as pd; import numpy as np

    def _ensure_list(x):
        if x is None: return []
        if isinstance(x, (list, tuple, set)): return list(x)
        return [x]

    def _gather_objectids(node_dict):
        ids = []
        if not isinstance(node_dict, dict): return ids
        if "objectid"  in node_dict: ids.extend(_ensure_list(node_dict["objectid"]))
        if "objectids" in node_dict: ids.extend(_ensure_list(node_dict["objectids"]))
        return [str(i).strip() for i in ids if i is not None and str(i).strip() != ""]

    def _dedup(seq):
        seen=set(); out=[]
        for x in seq:
            if x not in seen:
                out.append(x); seen.add(x)
        return out

    def _keep_odd_objectids(ids):
        odds = [i for i in ids if i.isdigit() and (int(i) % 2 == 1)]
        return odds if odds else ids

    def _normalize_one_location(name, loc, include_oid_in_name=True):
        nodes = (loc.get("nodes") if isinstance(loc, dict) else None) or {}
        all_ids=[]
        for _, node in nodes.items():
            all_ids.extend(_gather_objectids(node))
        if not all_ids and isinstance(loc, dict) and "objectid" in loc:
            all_ids = [str(loc["objectid"])]

        all_ids = _dedup([i for i in all_ids if i])
        kept_ids = _keep_odd_objectids(all_ids)

        name_out = name
        if include_oid_in_name and kept_ids:
            name_out = f"{name} [{','.join(kept_ids)}]"

        return {
            "name": name_out,
            "daytype": (loc.get("daytype") if isinstance(loc, dict) else None) or "0: All Days (M-Su)",
            "objectids": kept_ids,
        }

    def _normalize_input(aadt_locs):
        if isinstance(aadt_locs, pd.DataFrame) and {"name","daytype","objectids"}.issubset(aadt_locs.columns):
            recs = aadt_locs.to_dict(orient="records")
            for r in recs:
                r["objectids"] = _keep_odd_objectids(_ensure_list(r.get("objectids")))
            return recs
        if isinstance(aadt_locs, list) and aadt_locs and isinstance(aadt_locs[0], dict) and \
           {"name","daytype","objectids"}.issubset(aadt_locs[0].keys()):
            recs = []
            for r in aadt_locs:
                r = dict(r)
                r["objectids"] = _keep_odd_objectids(_ensure_list(r.get("objectids")))
                recs.append(r)
            return recs

        recs = []
        if isinstance(aadt_locs, dict):
            for nm, loc in aadt_locs.items():
                recs.append(_normalize_one_location(nm, loc))
            return recs

        if isinstance(aadt_locs, list):
            for item in aadt_locs:
                if not isinstance(item, dict):
                    continue
                if "nodes" in item:
                    nm = item.get("location_description") or item.get("name") or "UNKNOWN"
                    recs.append(_normalize_one_location(nm, item))
                elif "objectid" in item:
                    oid = str(item.get("objectid")).strip()
                    nm  = item.get("location_description") or item.get("name") or "UNKNOWN"
                    kept = _keep_odd_objectids([oid])
                    recs.append({
                        "name": f"{nm} [{','.join(kept)}]" if kept else nm,
                        "daytype": item.get("daytype", "0: All Days (M-Su)"),
                        "objectids": kept,
                    })
                else:
                    for nm, loc in item.items():
                        recs.append(_normalize_one_location(nm, loc))
        return recs

    def _traditional_aadt_for_ids(df_tc_in, obj_ids):
        """
        Default (use_parity=False): per-oid average of (ahead_aadt, back_aadt), then mean across oids.
        If use_parity=True: even->back_aadt, odd->ahead_aadt.
        """
        obj_ids = [str(x).strip() for x in (obj_ids or []) if str(x).strip()]
        if not obj_ids:
            return np.nan, np.nan, np.nan, 0

        sub = df_tc_in[df_tc_in["objectid"].astype(str).str.strip().isin(obj_ids)].copy()
        if sub.empty:
            return np.nan, np.nan, np.nan, 0

        if use_parity:
            vals = []
            for oid in obj_ids:
                row = sub[sub["objectid"].astype(str).str.strip() == oid]
                if row.empty:
                    continue
                v = row.iloc[0]["back_aadt"] if (oid.isdigit() and int(oid) % 2 == 0) else row.iloc[0]["ahead_aadt"]
                vals.append(pd.to_numeric(v, errors="coerce"))
            vals = pd.Series(vals, dtype="float64").dropna()
            if vals.empty: return np.nan, np.nan, np.nan, 0
            overall = float(vals.mean())
            return overall, np.nan, np.nan, int(vals.shape[0])

        # --- average ahead/back per objectid, then average across objectids ---
        sub["ahead_aadt"] = pd.to_numeric(sub.get("ahead_aadt"), errors="coerce")
        sub["back_aadt"]  = pd.to_numeric(sub.get("back_aadt"),  errors="coerce")

        # per-oid average: mean of available sides (ignore NaN)
        per_oid_avg = sub[["ahead_aadt","back_aadt"]].mean(axis=1, skipna=True)
        per_oid_avg = per_oid_avg.dropna()

        if per_oid_avg.empty:
            return np.nan, np.nan, np.nan, 0

        overall = float(per_oid_avg.mean())

        # side means (for reporting only)
        ahead_vals = sub["ahead_aadt"].dropna()
        back_vals  = sub["back_aadt"].dropna()
        mean_ahead = float(ahead_vals.mean()) if not ahead_vals.empty else np.nan
        mean_back  = float(back_vals.mean())  if not back_vals.empty  else np.nan
        count_used = int(per_oid_avg.shape[0])

        return overall, mean_ahead, mean_back, count_used

    # ---- main ----
    norm = _normalize_input(aadt_locations)
    tc_ids_all = set(df_tc["objectid"].astype(str).str.strip().unique())

    rows = []
    for loc in norm:
        obj_ids = [str(x).strip() for x in (loc.get("objectids") or []) if str(x).strip()]
        overall, mean_ahead, mean_back, n_found = _traditional_aadt_for_ids(df_tc, obj_ids)
        missing = [x for x in obj_ids if x not in tc_ids_all]

        rows.append({
            "location": loc.get("name"),
            "daytype":  loc.get("daytype"),
            "objectids": "|".join(obj_ids),
            "n_objectids": len(obj_ids),
            "n_found_in_tc": int(n_found),
            "missing_objectids": "|".join(missing) if missing else "",
            "traditional_ahead_mean": mean_ahead,
            "traditional_behind_mean": mean_back,
            "traditional_aadt": overall,
        })

    return pd.DataFrame(rows) if as_df else rows

In [17]:
# run step 1 - traditional aadt counts
trad_df = traditional_aadt_by_location(aadt_locations, df_tc, as_df=True)

In [18]:
# Export Step 1 as a CSV to take a look
trad_df.to_csv("step_1_traditional_aadt_by_location_99_d3.csv", index=False)

## Step 2, Non-Traditional AADT Summarizer

* **Goal:** For each locaiton, use its **StreetLight segment lists** ahead_zones/behind_zones (and also keep the "all" lists before de-dup for optional counts
* **How it works:**
    1. **Normalize locations** to get clean, de-duplicated ahead_zones/behind_zones (and also keep the "all" lists before de-dup for optional counts). 
    2. **Filter StreetLight rows** by daytype, daypart, and optional modeoftravel
    3. **Collapse StreetLight by segment:** groupby(zonename) to get a per-segment mean volume and a row count backing that mean
    4. For each location/side:
        * Pick which list to use for aggregation (**unique** segments for **all** before de-dup)
        * Pull the per-segment means for segments that are present; record any **missing** segments
        * **Aggregate within a side** using *agg* (**sum** by default; or *mean* if chosen).
    5. **Combine directions**: average the two sides when both exist otherwise take the side that exists. 
* **Outputs (per location)**: metadata (filters used), the zone list, **per-side values, overall non-traditional AADT, backing row counts,** and **segment presence/missing** diagnostics. 
* **Why this matters:** Produces a consistent, auditable **Non-Traditional baseline** to compare with the traditional AADT from Step 2.


In [19]:
# ------------------------------------------------------
# 2) Non-Traditional AADT Summarizer
# ------------------------------------------------------

def non_traditional_aadt_by_location(
    aadt_locations,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)",
    as_df=True,
    agg="sum",                       # per-side aggregation across segments: "sum" | "mean"
    segment_count_mode="unique",     # count unique zonenames per side ("unique") or all ("all")
    include_oid_in_name=True,
):
    """
    pseudo-plan:
      1) normalize mapping → keep per-direction (northbound/southbound) lists
      2) filter df_stl to daytype/daypart(/mode) and precompute per-zonename mean + row counts
      3) for each direction:
           side_a = agg(segments in ahead), side_b = agg(segments in behind)
           dir_total = average(side_a, side_b)  # use mean to avoid doubling a single direction
      4) location total = SUM of all dir_total (NB + SB)            <-- key change
      5) emit legacy side columns as NaN (optional to backfill later), plus rich debug fields
    """
    import numpy as np, pandas as pd, re, unicodedata

    # ---------- helpers ----------
    def _ensure_list(x):
        if x is None: return []
        if isinstance(x, (list, tuple, set)): return list(x)
        return [x]

    def _dedup(seq):
        seen=set(); out=[]
        for x in seq:
            if x not in seen:
                out.append(x); seen.add(x)
        return out

    def _clean_z(s):
        if s is None: return ""
        s = str(s)
        s = unicodedata.normalize("NFKC", s)
        s = re.sub(r"\s+", " ", s).strip()
        return s

    def _gather_zones(node_dict):
        ahead  = [_clean_z(z) for z in _ensure_list(node_dict.get("zonename_ahead", []))]
        behind = [_clean_z(z) for z in _ensure_list(node_dict.get("zonename_behind", []))]
        return ahead, behind

    def _gather_objectids(node_dict):
        ids=[]
        if isinstance(node_dict, dict):
            if "objectid"  in node_dict:  ids += _ensure_list(node_dict["objectid"])
            if "objectids" in node_dict:  ids += _ensure_list(node_dict["objectids"])
        return [str(i).strip() for i in ids if i is not None and str(i).strip()!=""]

    def _combine_sides(a, b):
        # average the two sides if both exist; else take the one that exists
        a = np.nan if a is None else a
        b = np.nan if b is None else b
        if pd.notna(a) and pd.notna(b): return (float(a)+float(b))/2.0
        if pd.notna(a): return float(a)
        if pd.notna(b): return float(b)
        return np.nan

    # ---------- normalize input (preserve directions) ----------
    def _normalize(aadt_locs):
        recs=[]
        # accept: dict; list with one giant dict (your pattern); or list of single-loc dicts
        items = [aadt_locs] if isinstance(aadt_locs, dict) else (aadt_locs if isinstance(aadt_locs, list) else [])
        for item in items:
            if not isinstance(item, dict): continue
            if "nodes" in item:
                # single location object
                nm = item.get("location_description") or item.get("name") or "UNKNOWN"
                nodes = item.get("nodes", {}) or {}
                oids=[]; dirs=[]
                for k,node in nodes.items():
                    a,b = _gather_zones(node)
                    oids += _gather_objectids(node)
                    direction = (_ensure_list(node.get("direction")) or [""])[0] or k
                    dirs.append({"direction":direction, "ahead":a, "behind":b})
                name_out = f"{nm} [{','.join(_dedup(oids))}]" if (include_oid_in_name and oids) else nm
                recs.append({"name": name_out, "daytype": item.get("daytype", daytype_filter), "dirs": dirs})
            else:
                # giant dict keyed by location names
                for nm,loc in item.items():
                    nodes = loc.get("nodes", {}) or {}
                    oids=[]; dirs=[]
                    for k,node in nodes.items():
                        a,b = _gather_zones(node)
                        oids += _gather_objectids(node)
                        direction = (_ensure_list(node.get("direction")) or [""])[0] or k
                        dirs.append({"direction":direction, "ahead":a, "behind":b})
                    name_out = f"{nm} [{','.join(_dedup(oids))}]" if (include_oid_in_name and oids) else nm
                    recs.append({"name": name_out, "daytype": loc.get("daytype", daytype_filter), "dirs": dirs})
        return recs

    # ---------- filter STL + precompute per-zonename means ----------
    must = {zonename_col, stl_volume_col, "daytype", "daypart"}
    miss = [c for c in must if c not in df_stl.columns]
    if miss:
        raise KeyError(f"df_stl missing columns: {miss}")

    filt = (df_stl["daytype"]==daytype_filter) & (df_stl["daypart"]==daypart_filter)
    if modeoftravel_filter and ("modeoftravel" in df_stl.columns):
        filt = filt & (df_stl["modeoftravel"]==modeoftravel_filter)

    stl = df_stl.loc[filt, [zonename_col, stl_volume_col]].copy()
    stl[zonename_col] = stl[zonename_col].map(_clean_z)
    stl[stl_volume_col] = pd.to_numeric(stl[stl_volume_col], errors="coerce")

    zone_group = stl.groupby(zonename_col)[stl_volume_col]
    zone_mean  = zone_group.mean()   # per-zonename AADT mean
    zone_rows  = zone_group.size()   # row counts backing that mean
    present    = set(zone_mean.index)

    def _side_val(zones, agg_local="sum", dedup_mode=segment_count_mode):
        # clean → (dedup if requested) → split present/missing → aggregate
        zs = [_clean_z(z) for z in _ensure_list(zones) if _clean_z(z)]
        zs = _dedup(zs) if dedup_mode=="unique" else zs
        if not zs: return (np.nan, 0, [], 0, 0)
        got  = [z for z in zs if z in present]
        miss = [z for z in zs if z not in present]
        vals = zone_mean.reindex(got).dropna()
        if not len(vals): return (np.nan, 0, miss, 0, len(zs))
        val  = float(vals.sum()) if agg_local=="sum" else float(vals.mean())
        nrows= int(zone_rows.reindex(got).fillna(0).sum())
        return (val, nrows, miss, len(got), len(zs))

    # ---------- compute per-location ----------
    rows=[]
    for rec in _normalize(aadt_locations):
        loc_name = rec["name"]
        loc_day  = rec.get("daytype", daytype_filter)

        # unions for legacy/readability
        ahead_union=[]; behind_union=[]
        ahead_rows_total=0; behind_rows_total=0
        listed_ahead_total=0; listed_behind_total=0
        present_ahead_total=0; present_behind_total=0
        miss_a_all=[]; miss_b_all=[]

        dir_totals=[]; dir_dbg=[]; dir_counts_dbg=[]; dir_missing_dbg=[]

        for d in rec.get("dirs", []):
            a_val, a_n, a_miss, a_present, a_listed = _side_val(d["ahead"], agg)
            b_val, b_n, b_miss, b_present, b_listed = _side_val(d["behind"], agg)

            dir_total = _combine_sides(a_val, b_val)  # per-direction total (avg of sides)
            dir_totals.append(dir_total)
            dir_dbg.append(f"{d['direction']}={'' if pd.isna(dir_total) else round(dir_total,3)}")
            dir_counts_dbg.append(f"{d['direction']}(ahead {a_present}/{a_listed}, behind {b_present}/{b_listed})")
            dir_missing_dbg.append(
                f"{d['direction']}_ahead_missing={','.join(_dedup(a_miss))};"
                f"{d['direction']}_behind_missing={','.join(_dedup(b_miss))}"
            )

            ahead_union  += _ensure_list(d["ahead"])
            behind_union += _ensure_list(d["behind"])
            ahead_rows_total  += a_n
            behind_rows_total += b_n
            listed_ahead_total  += a_listed
            listed_behind_total += b_listed
            present_ahead_total += a_present
            present_behind_total+= b_present
            if a_miss: miss_a_all += a_miss
            if b_miss: miss_b_all += b_miss

        # KEY: location total is SUM of directional totals (NB + SB)
        if any(pd.notna(x) for x in dir_totals):
            loc_total = float(pd.Series(dir_totals).dropna().sum())
        else:
            loc_total = np.nan

        rows.append({
            "location": loc_name,
            "daytype_expected": loc_day,
            "daytype_used": daytype_filter,
            "daypart_used": daypart_filter,
            "modeoftravel_used": modeoftravel_filter or "",

            # deduped unions across directions
            "ahead_zones": "|".join(_dedup([z for z in ahead_union if z])),
            "behind_zones": "|".join(_dedup([z for z in behind_union if z])),

            # leave side means blank (optional to compute later)
            "non_trad_ahead_mean": np.nan,
            "non_trad_behind_mean": np.nan,

            # TOTAL non-traditional AADT for the location (NB + SB)
            "non_trad_aadt": loc_total,

            # backing row counts (sum across dirs)
            "stl_ahead_rows": ahead_rows_total,
            "stl_behind_rows": behind_rows_total,

            # segment counts (sum across dirs)
            "listed_ahead_segments": listed_ahead_total,
            "listed_behind_segments": listed_behind_total,
            "present_ahead_segments": present_ahead_total,
            "present_behind_segments": present_behind_total,

            # missing (union across dirs)
            "missing_ahead_zones": "|".join(_dedup(miss_a_all)),
            "missing_behind_zones": "|".join(_dedup(miss_b_all)),

            # debug: per-direction totals & coverage
            # (name kept for backward-compat even though these are totals)
            "non_trad_dir_means": "|".join(dir_dbg),
            "dir_present_counts": "|".join(dir_counts_dbg),
            "dir_missing_zones": "|".join(dir_missing_dbg),
        })

    return pd.DataFrame(rows) if as_df else rows


In [20]:
# this will run the "non_traditional_aadt_by_location" function if  you have the raw nested structure:
stl_df = non_traditional_aadt_by_location(
    aadt_locations,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter="All Vehicles - StL All Vehicles Volume",  # or None
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)",
    as_df=True
)

In [21]:
# Export step 2 to a CSV
stl_df.to_csv("step_2_non_traditional_aadt_by_location_99_d3.csv", index=False)

### Step 3, Build the per-location comparison DataFrame
* **Joining the base location**, not the raw location

-----------------------

* Build inputs with traditional_aadt_by_location(...) and non_traditional_aadt_by_location(...).
* Normalize names to a **base location** by stripping trailing [...] objectids.
* **Merge on** location_base (merge_how="inner" by default; pass "outer" to keep one-sided locations).
* Unify suffixed columns so downstream aggregation sees a consistent schema.
* Collapse duplicates by location_base:
    * Traditional metrics: **mean** across duplicate rows.
    * StreetLight per-direction metrics: **sum** across duplicate rows.
    * Strings (zones, missing): **stable unique union.
    * IDs: **extract digits** and **stable-unique join** with "|"; recompute n_objectids.
* Recompute StreetLight overall: non_trad_aadt = avg(non_trad_ahead_sum, non_trad_behind_sum) (or the lone side if one is missing).
* Compute tce_percent = 100 * (non_trad_aadt - traditional_aadt) / traditional_aadt.
* Return a tidy, consistent column order, with display location set to the base name.

In [22]:
# ------------------------------------------------------
# 3) Build the per-location comparison DataFrame
# ------------------------------------------------------

def build_aadt_comparison_df(
    aadt_locations,
    df_tc,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)",
    merge_how="inner",  # "outer" to keep one-sided locations
):
    """
    Step 4 (concise, pseudo-code style)

    # plan
    # 1) build traditional side + non-traditional side
    # 2) merge on location_base (strip "[...]" OIDs)
    # 3) unify column names; aggregate
    # 4) prefer Step 3's non_trad_aadt; only backfill from side means if missing
    # 5) compute TCE
    """
    import re
    import numpy as np
    import pandas as pd

    # -- helpers --------------------------------------------------------------
    def _base_location_local(s: str) -> str:
        # drop trailing "[...]" OID decoration
        if not isinstance(s, str): 
            return ""
        return re.sub(r"\s*\[[^\]]+\]\s*$", "", s).strip()

    def _pick_col_local(df: pd.DataFrame, base: str):
        # pick the first present variant
        if base in df.columns:
            return base
        for suf in ("_trad", "_nt", "_x", "_y"):
            c = f"{base}{suf}"
            if c in df.columns:
                return c
        return None

    def _unify_cols(df, bases):
        # copy any found variant back into the base name
        for base in bases:
            c = _pick_col_local(df, base)
            if c and c != base:
                df[base] = df[c]

    def uniq_join(series):
        # pipe-join unique tokens across rows
        seen=set(); out=[]
        for s in series.dropna().astype(str):
            for tok in str(s).split("|"):
                tok = tok.strip()
                if tok and tok not in seen:
                    seen.add(tok); out.append(tok)
        return "|".join(out)

    def join_objectids(series):
        # pull digit runs, dedupe, pipe-join
        toks=[]
        for s in series.astype(str).fillna(""):
            toks.extend(re.findall(r"\d+", s))
        out=[]; seen=set()
        for t in toks:
            if t not in seen:
                seen.add(t); out.append(t)
        return "|".join(out)

    # -- 1) build sides -------------------------------------------------------
    trad_df = traditional_aadt_by_location(
        aadt_locations=aadt_locations,
        df_tc=df_tc,
        as_df=True
    )
    nt_df = non_traditional_aadt_by_location(
        aadt_locations=aadt_locations,
        df_stl=df_stl,
        daytype_filter=daytype_filter,
        daypart_filter=daypart_filter,
        modeoftravel_filter=modeoftravel_filter,
        zonename_col=zonename_col,
        stl_volume_col=stl_volume_col,
        as_df=True
    )

    # add base names (strip "[...]" to guarantee merges)
    trad_df["location_base"] = trad_df["location"].apply(_base_location_local)
    nt_df["location_base"]   = nt_df["location"].apply(_base_location_local)

    # -- 2) merge -------------------------------------------------------------
    merged = pd.merge(
        trad_df,
        nt_df,
        how=merge_how,
        on="location_base",
        suffixes=("_trad", "_nt")
    )

    # unify expected columns (pull _trad/_nt into base)
    _unify_cols(merged, [
        # ids/meta
        "objectids","n_objectids","n_found_in_tc",
        "daytype","daytype_expected","daytype_used","daypart_used","modeoftravel_used",
        # zones & missing
        "ahead_zones","behind_zones","missing_ahead_zones","missing_behind_zones",
        # STL counts
        "stl_ahead_rows","stl_behind_rows",
        "listed_ahead_segments","listed_behind_segments",
        "present_ahead_segments","present_behind_segments",
        # metrics
        "traditional_ahead_mean","traditional_behind_mean","traditional_aadt",
        "non_trad_ahead_mean","non_trad_behind_mean","non_trad_aadt",  # <-- include total
    ])

    # display name = base
    merged["location"] = merged["location_base"]

    # -- 3) collapse dup rows by location_base --------------------------------
    agg = {
        "traditional_ahead_mean": "mean",
        "traditional_behind_mean": "mean",
        "traditional_aadt": "mean",
        "non_trad_ahead_mean": "mean",     # was "sum" -> turned NaN into 0; fixed
        "non_trad_behind_mean": "mean",
        "non_trad_aadt": "mean",           # prefer Step 3 total; average if multiple rows
        "stl_ahead_rows": "sum",
        "stl_behind_rows": "sum",
        "listed_ahead_segments": "sum",
        "listed_behind_segments": "sum",
        "present_ahead_segments": "sum",
        "present_behind_segments": "sum",
        "ahead_zones": uniq_join,
        "behind_zones": uniq_join,
        "missing_ahead_zones": uniq_join,
        "missing_behind_zones": uniq_join,
        "objectids": join_objectids,
        "n_objectids": "sum",              # recomputed below anyway
        "n_found_in_tc": "sum",
        "daytype": "first",
        "daytype_expected": "first",
        "daytype_used": "first",
        "daypart_used": "first",
        "modeoftravel_used": "first",
    }
    agg = {k:v for k,v in agg.items() if k in merged.columns}

    out = (merged
           .groupby("location_base", as_index=False)
           .agg(agg)
           .rename(columns={"location_base":"location"}))

    # harden objectids; recompute counts from the pipe string
    if "objectids" in out.columns:
        out["objectids"] = out["objectids"].astype(str)
        out["n_objectids"] = out["objectids"].str.split(r"\|").apply(lambda xs: len([t for t in xs if t and t.strip()]))

    # -- 4) prefer Step 3 non_trad_aadt, backfill from sides if missing -------
    if {"non_trad_ahead_mean","non_trad_behind_mean"}.issubset(out.columns):
        if "non_trad_aadt" not in out.columns:
            out["non_trad_aadt"] = np.nan
        out["non_trad_aadt"] = pd.to_numeric(out["non_trad_aadt"], errors="coerce")

        a = pd.to_numeric(out["non_trad_ahead_mean"], errors="coerce")
        b = pd.to_numeric(out["non_trad_behind_mean"], errors="coerce")
        comp = np.where(a.notna() & b.notna(), (a + b) / 2.0,
                        np.where(a.notna(), a, b))
        # only fill where total is missing
        out["non_trad_aadt"] = out["non_trad_aadt"].where(out["non_trad_aadt"].notna(), comp)

    # -- 5) TCE ---------------------------------------------------------------
    if {"traditional_aadt","non_trad_aadt"}.issubset(out.columns):
        T = pd.to_numeric(out["traditional_aadt"], errors="coerce")
        N = pd.to_numeric(out["non_trad_aadt"], errors="coerce")
        out["tce_percent"] = np.where(T.notna() & (T != 0) & N.notna(), 100.0*(N - T)/T, np.nan)

    # -- 6) column order ------------------------------------------------------
    preferred_cols = [
        "location",
        "objectids","n_objectids","n_found_in_tc",
        "ahead_zones","behind_zones",
        "traditional_ahead_mean","traditional_behind_mean","traditional_aadt",
        "non_trad_ahead_mean","non_trad_behind_mean","non_trad_aadt",
        "tce_percent",
        "daytype","daytype_expected","daytype_used","daypart_used","modeoftravel_used",
        "stl_ahead_rows","stl_behind_rows",
        "listed_ahead_segments","listed_behind_segments",
        "present_ahead_segments","present_behind_segments",
        "missing_ahead_zones","missing_behind_zones",
    ]
    cols = [c for c in preferred_cols if c in out.columns]
    return out[cols].copy()

In [23]:
# 3.1, Build the combined comparison DataFrame
cmp_df = build_aadt_comparison_df(
    aadt_locations=aadt_locations,  # your dict/list structure
    df_tc=df_tc,                                 # Traffic Census dataframe
    df_stl=df_stl,                               # StreetLight dataframe
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,                    # e.g., "0: All Modes" if you need it
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)"
)

In [24]:
# 3.2, (Optional) sort by absolute TCE to see big deltas first
cmp_df = cmp_df.sort_values("tce_percent", key=lambda s: s.abs(), ascending=False)

In [25]:
# 3.3, Export to CSV 
cmp_df.to_csv("step_3_comparison_dataframe_99_d3.csv", index=False)

### Step 4 Collapse to one row per location

* **Goal**: Produce one row per physical location and compute the CI on TCE in a single pass.

-----------------------

* **How it collapses**:
    * Build a base location key by stripping trailing bracketed IDs
    * Traditional AADT metrics
    * Non-traditional side fields (non_trad_ahead_mean, non_trad_behind_mean): sum with NaN-safety (uses sum(min_count=1) so all-NaN stays NaN—no accidental zeros).
    * non_trad_aadt (the total): prefer the total from Step 4 (already NB+SB). If missing, backfill as the average of the side sums (or the lone side if only one exists).
    * Counts (stl_*_rows, listed_*_segments, present_*_segments): sum across duplicates.
    * String fields (zones, missing lists): stable unique union (pipe-joined).
    * ObjectIDs: extract digits only, stable-dedupe, pipe-join; recompute n_objectids.
    
* **Then recompute**:
    * tce_percent = 100 * (non_trad_aadt − traditional_aadt) / traditional_aadt, guarding divide-by-zero/NaN.

* ** Confidence Interval**
    * Build a clean 1-D series of tce_percent (drop NaN/±∞).
    * Optional filters:
        * cap_abs: drop rows with |tce| > cap_abs.
        * winsor_pct: winsorize tails (e.g., 0.01 → 1% each tail).
    * Compute mean TCE, std, SE, t-critical, 95% CI, t-statistic, p-value (two-sided), Cohen’s d.
    * Returns a one-row summary (ci_summary) plus the collapsed detail (detail_df). 

In [26]:
# ------------------------------------------------------
# 4) Collapse to one row per location & Run the Confidence Interval
# ------------------------------------------------------

def collapse_and_ci(
    cmp_df,
    confidence=0.95,
    tce_col="tce_percent",
    cap_abs=None,          # e.g., 500 caps |tce|
    winsor_pct=None        # e.g., 0.01 winsorizes 1% tails
):
    """
    1) Make a copy; derive base location name (strip trailing "[ids]") -> 'location_clean'.
    2) Group by 'location_clean' with safe aggregations:
       - traditional_* : mean (should match)
       - non_trad_*_mean : SUM with min_count=1 (all-NaN -> NaN, not 0)
       - non_trad_aadt : mean (prefer totals from Step 4; dups should agree)
       - counts : sum; strings : union pipe-join; objectids : extract digits + dedupe
    3) Recompute non_trad_aadt ONLY where missing:
       - backfill from side sums: avg(ahead_sum, behind_sum) or lone side
    4) Compute TCE = 100*(N - T)/T (guard divide-by-zero/NaN).
    5) Build one-row CI summary over tce_col with optional cap/winsorization.
    6) Return (detail_df, ci_summary_df).
    """
    import re
    import numpy as np
    import pandas as pd
    from scipy import stats

    df = cmp_df.copy()

    # -- 1) base location (defensive even if already clean) -------------------
    def _base_location(s: str) -> str:
        if not isinstance(s, str): return ""
        return re.sub(r"\s*\[[^\]]+\]\s*$", "", s).strip()

    df["location_clean"] = df["location"].astype(str).map(_base_location)

    # -- helpers ---------------------------------------------------------------
    def uniq_join(series):
        seen, out = set(), []
        for s in series.dropna().astype(str):
            for tok in s.split("|"):
                tok = tok.strip()
                if tok and tok not in seen:
                    seen.add(tok); out.append(tok)
        return "|".join(out)

    def join_objectids(series):
        toks = []
        for s in series.astype(str).fillna(""):
            toks.extend(re.findall(r"\d+", s))
        out, seen = [], set()
        for t in toks:
            if t not in seen:
                seen.add(t); out.append(t)
        return "|".join(out)

    def sum_min1(s):
        # numeric sum but return NaN if ALL are NaN
        return pd.to_numeric(s, errors="coerce").sum(min_count=1)

    # -- 2) collapse to one row per location ----------------------------------
    agg = {
        # traditional: mean
        "traditional_ahead_mean": "mean",
        "traditional_behind_mean": "mean",
        "traditional_aadt": "mean",

        # non-trad side fields: sum with NaN safety
        "non_trad_ahead_mean": sum_min1,
        "non_trad_behind_mean": sum_min1,

        # prefer totals from Step 4; dups should be equal → mean is fine
        "non_trad_aadt": "mean",

        # counts
        "stl_ahead_rows": "sum",
        "stl_behind_rows": "sum",
        "listed_ahead_segments": "sum",
        "listed_behind_segments": "sum",
        "present_ahead_segments": "sum",
        "present_behind_segments": "sum",

        # strings/meta
        "ahead_zones": uniq_join,
        "behind_zones": uniq_join,
        "missing_ahead_zones": uniq_join,
        "missing_behind_zones": uniq_join,
        "daytype": "first",
        "daytype_expected": "first",
        "daytype_used": "first",
        "daypart_used": "first",
        "modeoftravel_used": "first",

        # ids
        "objectids": join_objectids,
        "n_objectids": "sum",
        "n_found_in_tc": "sum",
    }
    agg = {k:v for k,v in agg.items() if k in df.columns}

    detail = (df
              .groupby("location_clean", as_index=False)
              .agg(agg)
              .rename(columns={"location_clean":"location"}))

    # recompute n_objectids from joined string (truth source)
    if "objectids" in detail.columns:
        detail["objectids"] = detail["objectids"].astype(str)
        detail["n_objectids"] = detail["objectids"].str.split(r"\|").apply(
            lambda xs: len([t for t in xs if t and t.strip()])
        )

    # -- 3) backfill non_trad_aadt ONLY if missing ----------------------------
    if {"non_trad_ahead_mean","non_trad_behind_mean"}.issubset(detail.columns):
        if "non_trad_aadt" not in detail.columns:
            detail["non_trad_aadt"] = np.nan
        detail["non_trad_aadt"] = pd.to_numeric(detail["non_trad_aadt"], errors="coerce")

        a = pd.to_numeric(detail["non_trad_ahead_mean"], errors="coerce")
        b = pd.to_numeric(detail["non_trad_behind_mean"], errors="coerce")
        comp = np.where(a.notna() & b.notna(), (a + b) / 2.0,
                        np.where(a.notna(), a, b))
        detail["non_trad_aadt"] = detail["non_trad_aadt"].where(detail["non_trad_aadt"].notna(), comp)

    # -- 4) TCE ----------------------------------------------------------------
    if {"traditional_aadt","non_trad_aadt"}.issubset(detail.columns):
        T = pd.to_numeric(detail["traditional_aadt"], errors="coerce")
        N = pd.to_numeric(detail["non_trad_aadt"], errors="coerce")
        detail[tce_col] = np.where(T.notna() & (T != 0) & N.notna(), 100.0*(N - T)/T, np.nan)

    # -- 5) CI summary over tce_col -------------------------------------------
    tces = pd.to_numeric(detail[tce_col], errors="coerce").replace([np.inf, -np.inf], np.nan).dropna()

    # optional absolute cap
    if cap_abs is not None and len(tces) > 0:
        tces = tces[np.abs(tces) <= float(cap_abs)]

    # optional winsorization
    if winsor_pct is not None and 0 < winsor_pct < 0.5 and len(tces) > 0:
        lo = float(tces.quantile(winsor_pct))
        hi = float(tces.quantile(1 - winsor_pct))
        tces = tces.clip(lower=lo, upper=hi)

    n = int(tces.shape[0])
    if n == 0:
        ci_df = pd.DataFrame([{
            "confidence": confidence,
            "tce_col": tce_col,
            "n": 0, "dof": None,
            "mean_tce": None, "std_tce": None, "se": None,
            "t_critical": None, "margin_of_error": None,
            "ci_lower": None, "ci_upper": None,
            "t_statistic": None, "p_value_two_sided": None, "cohens_d": None,
            "cap_abs": cap_abs, "winsor_pct": winsor_pct
        }])
        return detail, ci_df

    mean_tce = float(tces.mean())
    if n > 1:
        std_tce = float(tces.std(ddof=1))
        se = std_tce / np.sqrt(n) if std_tce > 0 else 0.0
        dof = n - 1
        if se > 0:
            tcrit = float(stats.t.ppf((1 + confidence) / 2.0, dof))
            moe = tcrit * se
            ci_lo, ci_hi = mean_tce - moe, mean_tce + moe
            t_stat = mean_tce / se
            p_val = float(2 * (1 - stats.t.cdf(abs(t_stat), dof)))
            cohens_d = mean_tce / std_tce if std_tce > 0 else None
        else:
            tcrit = moe = ci_lo = ci_hi = t_stat = p_val = cohens_d = None
    else:
        std_tce = se = None
        dof = None
        tcrit = moe = ci_lo = ci_hi = t_stat = p_val = cohens_d = None

    ci_df = pd.DataFrame([{
        "confidence": confidence,
        "tce_col": tce_col,
        "n": n, "dof": dof,
        "mean_tce": mean_tce, "std_tce": std_tce, "se": se,
        "t_critical": tcrit, "margin_of_error": moe,
        "ci_lower": ci_lo, "ci_upper": ci_hi,
        "t_statistic": t_stat, "p_value_two_sided": p_val, "cohens_d": cohens_d,
        "cap_abs": cap_abs, "winsor_pct": winsor_pct
    }])

    return detail, ci_df




In [27]:
# 4.1, Run the Confidence Interval
detail_df, ci_summary = collapse_and_ci(cmp_df, confidence=0.95)

In [28]:
# 4.2, View the results of the confidence interval
print(ci_summary)

   confidence      tce_col   n  dof  mean_tce    std_tce        se  \
0        0.95  tce_percent  49   48 -2.758799  22.203273  3.171896   

   t_critical  margin_of_error  ci_lower  ci_upper  t_statistic  \
0    2.010635         6.377525 -9.136323  3.618726    -0.869763   

   p_value_two_sided  cohens_d cap_abs winsor_pct  
0           0.388759 -0.124252    None       None  


In [29]:
# 4.3 Export to CSV 
detail_df.to_csv("step_4_summary_99_d3.csv", index=False)

## Step 5, Analyst's Analysis

In [30]:
r = ci_summary.iloc[0]  # one-row summary

mean_tce     = r["mean_tce"]
ci_lower     = r["ci_lower"]
ci_upper     = r["ci_upper"]
t_critical   = r["t_critical"]
t_statistic  = r["t_statistic"]

# pretty print (guard NaNs)
fmt = lambda x: "NA" if pd.isna(x) else f"{x:.3f}"
print("Mean TCE:", fmt(mean_tce))
print("95% Confidence Interval:", (fmt(ci_lower), fmt(ci_upper)))
print("t-test statistic:", fmt(t_statistic))
print("t-critical:", fmt(t_critical))

Mean TCE: -2.759
95% Confidence Interval: ('-9.136', '3.619')
t-test statistic: -0.870
t-critical: 2.011


### Mean TCE: -2.134
Traffic Count Error (TCE)
* On average across SR-99 (District 3), Non-Traditional AADT (StreetLight) is about 2.1% lower than Traditional AADT (Traffic Census).

### 95% Confidence Interval (-11.3%, 7.1%)
* We’re 95% confident the true average difference lies between 11.3% lower and 7.1% higher.
* Because this range includes zero, there’s no clear corridor-level bias—StreetLight isn’t consistently above or below Traffic Census on average.

### T-Test Statistic (-0.524) 
* The observed average difference is ~0.52 standard errors below zero (t == −0.524), far from the ±2 threshold for 95% significance.

### Summary
* On SR-99 (District 3), StreetLight AADT averages about 3.6% lower than the Traffic Census.
* With 95% confidence, the true average difference could be anywhere from 10.3% lower to 3.2% higher.
* Because that range includes zero and the t-stat is −1.06 (well below the ±2 needed at 95%), the average difference is not statistically significant.